# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [16]:

import os

def buscar_palabra(carpeta, palabra):
    documentos = os.listdir(carpeta)
    resultados = {}

    for doc in documentos:
        if doc.endswith('.txt'):
            ruta = os.path.join(carpeta, doc)
            archivo = open(ruta, 'r', encoding='utf-8')
            lineas = archivo.readlines()
            archivo.close()

            i = 0
            cantidad = 0
            documentos_encontrados = []
            for linea in lineas:
                if palabra.lower() in linea.lower():
                    documentos_encontrados.append("Documento " + str(i + 1))
                    cantidad = cantidad + linea.lower().count(palabra.lower())
                i = i + 1

            if len(documentos_encontrados) > 0:
                resultados[doc] = {
                    'documentos': documentos_encontrados,
                    'cantidad': cantidad
                }

    if len(resultados) > 0:
        print("La palabra '" + palabra + "' aparece en los siguientes documentos:")
        for doc in resultados:
            print("En el archivo " + doc + " aparece " + str(resultados[doc]['cantidad']) + " veces")
            for documento in resultados[doc]['documentos']:
                print("  - " + documento)
    else:
        print("La palabra '" + palabra + "' no se encuentra en ningún documento.")

palabra = input("Ingrese la palabra que desea buscar: ")
carpeta = "/kaggle/input/corpus"

buscar_palabra(carpeta, palabra)




Ingrese la palabra que desea buscar:  quito


La palabra 'quito' aparece en los siguientes documentos:
En el archivo 01_corpus_turismo_500.txt aparece 20 veces
  - Documento 4
  - Documento 16
  - Documento 17
  - Documento 39
  - Documento 43
  - Documento 73
  - Documento 75
  - Documento 89
  - Documento 104
  - Documento 136
  - Documento 201
  - Documento 213
  - Documento 238
  - Documento 260
  - Documento 291
  - Documento 325
  - Documento 355
  - Documento 391
  - Documento 469
  - Documento 487
En el archivo 01_corpus_turismo.txt aparece 1 veces
  - Documento 3


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [17]:
import os

def construir_indice_invertido(carpeta):
    documentos = os.listdir(carpeta)
    indice_invertido = {}

    for doc in documentos:
        if doc.endswith('.txt'):
            ruta = os.path.join(carpeta, doc)
            archivo = open(ruta, 'r', encoding='utf-8')
            lineas = archivo.readlines()
            archivo.close()

            i = 0
            for linea in lineas:
                palabras = linea.lower().split()
                for palabra in palabras:
                    palabra = palabra.strip('.,;:¡!¿?"()')  
                    if palabra != '':
                        if palabra not in indice_invertido:
                            indice_invertido[palabra] = []
                        indice_invertido[palabra].append((doc, i + 1))  
                i = i + 1
    return indice_invertido

def buscar_palabra_indice(indice, palabra):
    palabra = palabra.lower()
    if palabra in indice:
        apariciones = indice[palabra]
        resultados = {}
        for doc, documento_num in apariciones:
            if doc not in resultados:
                resultados[doc] = []
            resultados[doc].append("Documento " + str(documento_num))

        print("La palabra '" + palabra + "' aparece en los siguientes documentos:")
        for doc in resultados:
            print("En el archivo " + doc + " aparece " + str(len(resultados[doc])) + " veces")
            for documento in resultados[doc]:
                print("  - " + documento)
    else:
        print("La palabra '" + palabra + "' no se encuentra en ningún documento.")

carpeta = "/kaggle/input/corpus"
indice_invertido = construir_indice_invertido(carpeta)

palabra = input("Ingrese la palabra que desea buscar: ")
buscar_palabra_indice(indice_invertido, palabra)




Ingrese la palabra que desea buscar:  turismo


La palabra 'turismo' aparece en los siguientes documentos:
En el archivo 01_corpus_turismo_500.txt aparece 36 veces
  - Documento 5
  - Documento 22
  - Documento 30
  - Documento 37
  - Documento 41
  - Documento 45
  - Documento 76
  - Documento 109
  - Documento 110
  - Documento 122
  - Documento 127
  - Documento 130
  - Documento 156
  - Documento 163
  - Documento 168
  - Documento 200
  - Documento 225
  - Documento 231
  - Documento 246
  - Documento 254
  - Documento 258
  - Documento 280
  - Documento 307
  - Documento 319
  - Documento 324
  - Documento 333
  - Documento 352
  - Documento 371
  - Documento 379
  - Documento 383
  - Documento 386
  - Documento 442
  - Documento 459
  - Documento 486
  - Documento 491
  - Documento 492
En el archivo 01_corpus_turismo.txt aparece 1 veces
  - Documento 1


## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

In [18]:
import os
import time
import pandas as pd

#Búsqueda Lineal
def buscar_palabra_lineal(carpeta, palabra):
    documentos = os.listdir(carpeta)
    resultados = {}

    palabra = palabra.lower()

    for doc in documentos:
        if doc.endswith('.txt'):
            ruta = os.path.join(carpeta, doc)
            with open(ruta, 'r', encoding='utf-8') as archivo:
                lineas = archivo.readlines()

            documentos_encontrados = []
            cantidad = 0
            for i, linea in enumerate(lineas):
                palabras_linea = linea.lower().split()
                palabras_linea = [p.strip('.,;:¡!¿?"()') for p in palabras_linea]

                if palabra in palabras_linea:
                    documentos_encontrados.append(f"Documento {i + 1}")
                    cantidad += palabras_linea.count(palabra)

            if documentos_encontrados:
                resultados[doc] = {
                    'documentos': documentos_encontrados,
                    'cantidad': cantidad
                }

    return resultados

#Índice Invertido
def construir_indice_invertido(carpeta):
    documentos = os.listdir(carpeta)
    indice_invertido = {}

    for doc in documentos:
        if doc.endswith('.txt'):
            ruta = os.path.join(carpeta, doc)
            with open(ruta, 'r', encoding='utf-8') as archivo:
                lineas = archivo.readlines()

            for i, linea in enumerate(lineas):
                palabras = linea.lower().split()
                palabras = [p.strip('.,;:¡!¿?"()') for p in palabras]

                for palabra in palabras:
                    if palabra:
                        if palabra not in indice_invertido:
                            indice_invertido[palabra] = []
                        indice_invertido[palabra].append((doc, i + 1))

    return indice_invertido

def buscar_palabra_indice(indice, palabra):
    palabra = palabra.lower()
    resultados = {}

    if palabra in indice:
        apariciones = indice[palabra]
        for doc, num_doc in apariciones:
            if doc not in resultados:
                resultados[doc] = {'documentos': [], 'cantidad': 0}
            resultados[doc]['documentos'].append(f"Documento {num_doc}")
            resultados[doc]['cantidad'] += 1

    return resultados

#Resultados
def imprimir_resultados(resultados, metodo):
    if resultados:
        print(f"\nLa palabra usando {metodo} aparece en:")
        for doc in resultados:
            print(f"En el archivo {doc}: {resultados[doc]['cantidad']} veces")
            for documento in resultados[doc]['documentos']:
                print(f"  - {documento}")
    else:
        print(f"\nLa palabra no se encontró usando {metodo}.")

#Main
if __name__ == "__main__":
    carpeta = "/kaggle/input/corpus"
    palabra = input("Ingrese la palabra que desea buscar: ")

    # Búsqueda Lineal
    inicio_lineal = time.time()
    resultados_lineal = buscar_palabra_lineal(carpeta, palabra)
    fin_lineal = time.time()
    tiempo_lineal = fin_lineal - inicio_lineal
    imprimir_resultados(resultados_lineal, "búsqueda lineal")

    # Índice Invertido
    inicio_indice = time.time()
    indice = construir_indice_invertido(carpeta)
    resultados_indice = buscar_palabra_indice(indice, palabra)
    fin_indice = time.time()
    tiempo_indice = fin_indice - inicio_indice
    imprimir_resultados(resultados_indice, "índice invertido")

    # Tabla Comparativa
    print("\nComparativa de tiempo de ejecución (en segundos):\n")
    tabla = pd.DataFrame({
        "Método": ["Búsqueda Lineal", "Índice Invertido"],
        "Tiempo (s)": [tiempo_lineal, tiempo_indice]
    })
    print(tabla.to_string(index=False))


Ingrese la palabra que desea buscar:  quito



La palabra usando búsqueda lineal aparece en:
En el archivo 01_corpus_turismo_500.txt: 20 veces
  - Documento 4
  - Documento 16
  - Documento 17
  - Documento 39
  - Documento 43
  - Documento 73
  - Documento 75
  - Documento 89
  - Documento 104
  - Documento 136
  - Documento 201
  - Documento 213
  - Documento 238
  - Documento 260
  - Documento 291
  - Documento 325
  - Documento 355
  - Documento 391
  - Documento 469
  - Documento 487
En el archivo 01_corpus_turismo.txt: 1 veces
  - Documento 3

La palabra usando índice invertido aparece en:
En el archivo 01_corpus_turismo_500.txt: 20 veces
  - Documento 4
  - Documento 16
  - Documento 17
  - Documento 39
  - Documento 43
  - Documento 73
  - Documento 75
  - Documento 89
  - Documento 104
  - Documento 136
  - Documento 201
  - Documento 213
  - Documento 238
  - Documento 260
  - Documento 291
  - Documento 325
  - Documento 355
  - Documento 391
  - Documento 469
  - Documento 487
En el archivo 01_corpus_turismo.txt: 1 vec

## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_

In [25]:
import os
import time

def construir_indice_invertido(carpeta):
    documentos = os.listdir(carpeta)
    indice_invertido = {}

    for doc in documentos:
        if doc.endswith('.txt'):
            ruta = os.path.join(carpeta, doc)
            archivo = open(ruta, 'r', encoding='utf-8')
            lineas = archivo.readlines()
            archivo.close()

            i = 0
            for linea in lineas:
                palabras = linea.lower().split()  # Convertir todo a minúsculas
                for palabra in palabras:
                    palabra = palabra.strip('.,;:¡!¿?"()')  
                    if palabra != '':
                        if palabra not in indice_invertido:
                            indice_invertido[palabra] = []
                        indice_invertido[palabra].append((doc, i + 1))  # Guardar línea donde aparece la palabra
                i += 1
    return indice_invertido

def buscar_palabras_indice(indice, palabras):
    palabras = [palabra.lower() for palabra in palabras]  # Convertir a minúsculas
    resultados = {}

    for palabra in palabras:
        if palabra in indice:
            apariciones = indice[palabra]
            for doc, documento_num in apariciones:
                if doc not in resultados:
                    resultados[doc] = []
                resultados[doc].append(f"Palabra '{palabra}' en documento {documento_num}")
        else:
            resultados[palabra] = "No se encuentra en ningún documento."

    for doc, detalles in resultados.items():
        if isinstance(detalles, list):
            print(f"En el archivo {doc}:")
            for detalle in detalles:
                print(f"  - {detalle}")
        else:
            print(f"El término '{doc}' no se encuentra en ningún documento.")

# Calcular Speedup
def calcular_speedup(carpeta, palabras):
    inicio_construccion = time.time()
    indice_invertido = construir_indice_invertido(carpeta)
    fin_construccion = time.time()

    print(f"Tiempo para construir el índice: {fin_construccion - inicio_construccion} segundos.")

    inicio_busqueda = time.time()
    buscar_palabras_indice(indice_invertido, palabras)
    fin_busqueda = time.time()

    print(f"Tiempo para realizar la búsqueda: {fin_busqueda - inicio_busqueda} segundos.")
    print(f"Speedup (construcción + búsqueda): {fin_busqueda - inicio_construccion} segundos.")

carpeta = "/kaggle/input/corpus"
palabras_a_buscar = input("Ingrese las palabras que desea buscar, separadas por espacio: ").split()

calcular_speedup(carpeta, palabras_a_buscar)


Ingrese las palabras que desea buscar, separadas por espacio:  quito turismo playas


Tiempo para construir el índice: 0.0065152645111083984 segundos.
En el archivo 01_corpus_turismo_500.txt:
  - Palabra 'quito' en documento 4
  - Palabra 'quito' en documento 16
  - Palabra 'quito' en documento 17
  - Palabra 'quito' en documento 39
  - Palabra 'quito' en documento 43
  - Palabra 'quito' en documento 73
  - Palabra 'quito' en documento 75
  - Palabra 'quito' en documento 89
  - Palabra 'quito' en documento 104
  - Palabra 'quito' en documento 136
  - Palabra 'quito' en documento 201
  - Palabra 'quito' en documento 213
  - Palabra 'quito' en documento 238
  - Palabra 'quito' en documento 260
  - Palabra 'quito' en documento 291
  - Palabra 'quito' en documento 325
  - Palabra 'quito' en documento 355
  - Palabra 'quito' en documento 391
  - Palabra 'quito' en documento 469
  - Palabra 'quito' en documento 487
  - Palabra 'turismo' en documento 5
  - Palabra 'turismo' en documento 22
  - Palabra 'turismo' en documento 30
  - Palabra 'turismo' en documento 37
  - Palabra 